In [ ]:
### Double chain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat


In [2]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat


In [3]:
real_chain = {"recipe": chef_chain} | veg_chain
real_chain.invoke({
    "cuisine": "Korean"
})

AIMessage(content="For a vegetarian version of Bibimbap, you can easily substitute the beef with tofu or tempeh. Here's how you can prepare the tofu or tempeh as a replacement for the beef in this recipe:\n\n**Tofu:**\n1. Use firm or extra firm tofu for this recipe. Press the tofu to remove excess water by wrapping it in a clean kitchen towel or paper towels and placing a heavy object on top for about 15-30 minutes.\n2. Cut the tofu into thin slices or cubes, depending on your preference.\n3. Marinate the tofu in a mixture of soy sauce, sesame oil, and a pinch of black pepper for at least 30 minutes to allow the flavors to infuse.\n4. Cook the marinated tofu in a pan over high heat until browned and slightly crispy on the outside.\n\n**Tempeh:**\n1. Cut the tempeh into thin slices or cubes.\n2. Steam the tempeh for about 10 minutes to soften it and remove any bitterness.\n3. Marinate the tempeh in a mixture of soy sauce, sesame oil, and a pinch of black pepper for at least 30 minutes.\

In [ ]:
### PromptTemplate + FewShotChatMessagePromptTemplate + LengthBasedExampleSelector + RandomExampleSelector
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate 
from langchain.prompts.example_selector import LengthBasedExampleSelector 
from langchain.prompts.example_selector.base import BaseExampleSelector



chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(examples)]


example_prompt = PromptTemplate.from_template(
    "Human: {question}\nAi: {answer}"
)

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Greece")

'Human: What do you know about Italy?\nAi: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nWhat do you know about Greece?'

In [ ]:
### ChatPromptTemplate + FewShotChatMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


fewShot_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=fewShot_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert"),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Korea"})


        I know this:
        There are two Koreas - North Korea and South Korea
        Capital of South Korea: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won
        

AIMessageChunk(content='\n        I know this:\n        There are two Koreas - North Korea and South Korea\n        Capital of South Korea: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won\n        ')

In [ ]:
### prompt.json + prompt.yaml 
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt = load_prompt("./prompt.json") #여기만 .yaml로 바꾸면됨
prompt.format(country="Germany")



'What is the capital of Germany'

In [ ]:
### PipelinePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompts = [
    ("intro", intro), #final prompt안에 있는 이름, 해당하는 prompt 이룸순으로
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

In [ ]:
### set_llm_cache + set_debug
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(InMemoryCache()) ### => memonry cache
set_llm_cache(SQLiteCache("My database")) ### => database cache
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chat.predict("I wanna make intelligent web product. How can I?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: I wanna make intelligent web product. How can I?"
  ]
}
Creating an intelligent web product involves a combination of technical skills, creativity, and strategic thinking. Here are some steps you can take to create an intelligent web product:

1. Define your target audience: Understand who your target users are and what their needs and pain points are. This will help you tailor your product to meet their specific requirements.

2. Conduct market research: Research the competitive landscape and identify gaps in the market that your product can fill. This will help you position your product effectively and differentiate it from competitors.

3. Develop a clear product vision: Define the problem your product solves and the value it provides to users. This will guide the development process and ensure that your product meets the needs of your target audience.

4. Use intelligent technologies: Incorpora

'Creating an intelligent web product involves a combination of technical skills, creativity, and strategic thinking. Here are some steps you can take to create an intelligent web product:\n\n1. Define your target audience: Understand who your target users are and what their needs and pain points are. This will help you tailor your product to meet their specific requirements.\n\n2. Conduct market research: Research the competitive landscape and identify gaps in the market that your product can fill. This will help you position your product effectively and differentiate it from competitors.\n\n3. Develop a clear product vision: Define the problem your product solves and the value it provides to users. This will guide the development process and ensure that your product meets the needs of your target audience.\n\n4. Use intelligent technologies: Incorporate artificial intelligence, machine learning, and other intelligent technologies into your product to enhance its functionality and prov

In [25]:
chat.predict("I wanna make intelligent web product. How can I?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: I wanna make intelligent web product. How can I?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Creating an intelligent web product involves a combination of technical skills, creativity, and strategic thinking. Here are some steps you can take to create an intelligent web product:\n\n1. Identify a problem or need: Start by identifying a specific problem or need that your web product can address. Conduct market research and gather feedback from potential users to understand their pain points and preferences.\n\n2. Define your target audience: Clearly define your target audience and understand their demographics, behaviors, and preferences. This will help you tailor your product to meet their needs and expectations.\n\n3. Develop a concept: Once you have identified a problem and defined your target audience, develop a concept f

'Creating an intelligent web product involves a combination of technical skills, creativity, and strategic thinking. Here are some steps you can take to create an intelligent web product:\n\n1. Identify a problem or need: Start by identifying a specific problem or need that your web product can address. Conduct market research and gather feedback from potential users to understand their pain points and preferences.\n\n2. Define your target audience: Clearly define your target audience and understand their demographics, behaviors, and preferences. This will help you tailor your product to meet their needs and expectations.\n\n3. Develop a concept: Once you have identified a problem and defined your target audience, develop a concept for your intelligent web product. This could involve creating wireframes, mockups, and prototypes to visualize your idea.\n\n4. Choose the right technology: Select the appropriate technology stack for your web product based on your requirements and budget. C

In [8]:
### Usage
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
    model="gpt-3.5-turbo"
)

with get_openai_callback() as usage:
    a = chat.predict("오늘 nasdaq시장은 어때보여?")
    print(a)
    print(usage)

죄송합니다, 저는 현재 실시간으로 시장 정보를 제공할 수 없습니다. 하지만 일반적으로 Nasdaq 시장은 기술 기업들이 많이 상장되어 있어 변동성이 크고 활발한 시장으로 알려져 있습니다. 투자를 고려하신다면 전문가의 조언을 듣고 신중한 판단을 하는 것이 중요합니다.
Tokens Used: 146
	Prompt Tokens: 22
	Completion Tokens: 124
Successful Requests: 1
Total Cost (USD): $0.000281


In [ ]:
### Model save
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

chat = OpenAI(
    temperature=0.1,
)

chat.save("model.json")
chat.load_llm("model.json")